In [1]:
import sys
!{sys.executable} -m pip install yfinance

  Using cached yfinance-0.2.41-py2.py3-none-any.whl.metadata (11 kB)
  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.0.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached lxml-5.3.0-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached frozendict-2.4.4-py312-none-any.whl.metadata (23 kB)
  Using cached peewee-3.17.6.tar.gz (3.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Us

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import yfinance as yf
from datetime import datetime
import time

In [3]:
# Base class for trading strategies
class TradingStrategy:
    def __init__(self, name):
        self.__name = name

    def generate_signal(self, price_data):
        print("This method should be overridden by subclasses")
        return "hold"
    
    @property
    def name(self):
        return self.__name

In [4]:
# Simple Moving Average Strategy
class SMAStrategy(TradingStrategy):
    def __init__(self, short_window, long_window):
        super().__init__("Simple Moving Average Strategy")
        self.__short_window = short_window
        self.__long_window = long_window

    def generate_signal(self, price_data):
        if len(price_data) < self.__long_window:
            return "hold"
        
        short_avg = sum(price_data[-self.__short_window:]) / self.__short_window
        long_avg = sum(price_data[-self.__long_window:]) / self.__long_window
        
        if short_avg > long_avg:
            return "buy"
        elif short_avg < long_avg:
            return "sell"
        else:
            return "hold"

    @property
    def short_window(self):
        return self.__short_window

    @property
    def long_window(self):
        return self.__long_window

In [5]:
# Trade management class
class Trade:
    def __init__(self, strategy_name, signal, amount):
        self.__strategy_name = strategy_name
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()

    def execute(self):
        print(f"Executing {self.__signal} for {self.__amount} units using {self.__strategy_name} at {self.__timestamp}")

    @property
    def strategy_name(self):
        return self.__strategy_name

    @property
    def signal(self):
        return self.__signal

    @property
    def amount(self):
        return self.__amount

    @property
    def timestamp(self):
        return self.__timestamp

In [6]:
# Mock trading API
class MockTradingAPI:
    def __init__(self, balance):
        self.__balance = balance

    def place_order(self, trade, price):
        if trade.signal == "buy" and self.__balance >= trade.amount * price:
            self.__balance -= trade.amount * price
            print(f"Placed buy order for {trade.amount} units at {price}. Remaining balance: {self.__balance}")
        elif trade.signal == "sell":
            self.__balance += trade.amount * price
            print(f"Placed sell order for {trade.amount} units at {price}. Remaining balance: {self.__balance}")
        else:
            print("Insufficient balance or invalid signal.")

    def get_balance(self):
        return self.__balance

In [7]:
# Main trading system
class TradingSystem:
    def __init__(self, api, strategy, symbol):
        self.__api = api
        self.__strategy = strategy
        self.__symbol = symbol
        self.__price_data = []

    def fetch_price_data(self):
        data = yf.download(tickers=self.__symbol, period='1d', interval='1m')
        if not data.empty:
            price = data['Close'].iloc[-1]
            self.__price_data.append(price)
            if len(self.__price_data) > self.__strategy.long_window:
                self.__price_data.pop(0)
            print(f"Fetched new price data: {price}")
        else:
            print("No data fetched")

    def run(self):
        self.fetch_price_data()
        signal = self.__strategy.generate_signal(self.__price_data)
        print(f"Generated signal: {signal}")
        if signal in ["buy", "sell"]:
            trade = Trade(self.__strategy.name, signal, 1)
            trade.execute()
            self.__api.place_order(trade, self.__price_data[-1])

    @property
    def api(self):
        return self.__api

    @property
    def strategy(self):
        return self.__strategy

    @property
    def symbol(self):
        return self.__symbol

    @property
    def price_data(self):
        return self.__price_data

In [8]:
# Example usage
if __name__ == "__main__":
    symbol = 'AAPL'

    api = MockTradingAPI(balance=10000)
    strategy = SMAStrategy(short_window=3, long_window=5)
    system = TradingSystem(api, strategy, symbol)

    for _ in range(10):
        system.run()
        print(f"Remaining balance: {api.get_balance()}")
        time.sleep(60)

[*********************100%%**********************]  1 of 1 completed


Fetched new price data: 221.27000427246094
Generated signal: hold
Remaining balance: 10000


[*********************100%%**********************]  1 of 1 completed


Fetched new price data: 221.27000427246094
Generated signal: hold
Remaining balance: 10000


[*********************100%%**********************]  1 of 1 completed


Fetched new price data: 221.27000427246094
Generated signal: hold
Remaining balance: 10000
